# OpenAI 

In [2]:
import getpass
import openai

client = openai.OpenAI(api_key=getpass.getpass('Enter your API key: '))

In [3]:
chat_completion = client.chat.completions.create(messages=[
    {
        "role": "user",
        "content": "what's my favorite color?",
    },
],
    model='gpt-4o'
)

In [7]:
print(chat_completion.choices[0].message.content)

I don't have access to personal information about you unless you share it with me. So, I don't know your favorite color. If you'd like to tell me, I'd be happy to chat about it!


## RAG
Let's add RAG to our interactions
We'll store documents in a vector database (ChromaDB) and we'll find 
documents most similar to our query to pass to LLM as an extra context

In [93]:
import chromadb

db = chromadb.Client()
collection = db.get_or_create_collection("my_documents")
collection.add(
    # under the hood chromadb will convert documents to embeddings (float vectors)
    # for that it downloads 79Mb transformer model 
    documents=["my favorite color is blue", "my favorite movies are Glengarry Glen Ross and Godfather part 2"],
    ids=["color", "movies"],
)

from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

# OpenAI provides embedding support in its API, we can use them instead
embedding_function = OpenAIEmbeddingFunction(
    api_key=client.api_key, model_name='text-embedding-3-large')
openai_collection = db.get_or_create_collection("openai_documents", embedding_function=embedding_function)
openai_collection.add(
    documents=["my favorite color is blue", "my favorite movies are Glengarry Glen Ross and Godfather part 2"],
    ids=["color", "movies"]
)

Insert of existing embedding ID: color
Insert of existing embedding ID: movies
Add of existing embedding ID: color
Add of existing embedding ID: movies


In [17]:
# chroma uses some measure to find the most suitable document
chroma_resp = collection.query(query_texts=["what is my favorite color?"], n_results=1)

In [18]:
print(chroma_resp['documents'][0][0])

my favorite color is blue


In [95]:
def query_with_context(question, coll):
    r = coll.query(query_texts=[question], n_results=1)
    context = r['documents'][0][0]

    completion = client.chat.completions.create(messages=[
        {
            "role": "system",
            "content": f"Give a short answer to a question. Extra context: {context}",
        },
        {
            "role": "user",
            "content": question,
        },
    ],
        model='gpt-4o'
    )
    return completion.choices[0].message.content

In [96]:
query_with_context("what is my favorite color?", collection)

'Blue.'

We can use OpenAI embeddings with pretty much the same result

In [98]:
query_with_context("what is my favorite color?", openai_collection)

'Your favorite color is blue.'

In [29]:
query_with_context("which actor stars in my favorite movies?")

'Al Pacino.'

## ReAct
Let's implement ReAct

In [31]:
query_with_context("what's the uppercased result of frobnication of 'openai python api'?")

"FROBNICATION OF 'OPENAI PYTHON API'"

In [55]:
import collections

# we need to store chat_history, as api has no "memory"
chat_history = collections.deque(maxlen=20)
chat_history.append({
    "role": "system",
    "content": """Give a short answer to a question.
        Here's are some useful tools at your disposal:
        * frobnicate(string) -> string
        * booya(int, int) -> int
        
        If you think that these tools can help you, then you should respond with valid json:
        {"tool": <tool_name>, "args": <args>}
        
        I'll call the tool in my next response and provide the result of the call as valid json: 
        {"result": <result>} 
        You can use this result to answer the original question.
        """,
})
chat_history.append({
    "role": "user",
    "content": "what's the uppercased result of frobnication of 'openai python api'?"
})
response = client.chat.completions.create(messages=chat_history, model='gpt-4o')

In [56]:
chat_history.append({"role": "assistant", "content": response.choices[0].message.content})

In [57]:
chat_history[-1]

{'role': 'assistant',
 'content': '{"tool": "frobnicate", "args": ["openai python api"]}'}

In [59]:
import json

request = json.loads(chat_history[-1]['content'])
print(request)

{'tool': 'frobnicate', 'args': ['openai python api']}


In [61]:
# here we can get a tool name from parsed json and call some function based on e.g. dict lookup
# let's just fake it
chat_history.append({"role": "user", "content": json.dumps({"result": "it's awesome"})})

In [62]:
response = client.chat.completions.create(messages=chat_history, model='gpt-4o')

In [63]:
print(response.choices[0].message.content)

IT'S AWESOME


ReAct works, LLM did all the job, even uppercased the result.

## Image generation

In [65]:
response = client.images.generate(
    model='dall-e-3',
    prompt='bullet dodging scene from the matrix but with penguin and elephant',
    size='1024x1024',
    quality='standard',
    # generate 1 image, actually dall-e-3 can do only 1 image per api call
    n=1,
    response_format='b64_json',
)

In [82]:
import base64

b64_image = response.data[0].b64_json
decoded = base64.b64decode(b64_image)

In [86]:
import pathlib

pathlib.Path('genai_image.png').write_bytes(decoded)

3162696

![image](./genai_image.png)

## Image description

In [83]:
response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "what's in this image? Give a short reply"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{b64_image}",
                    }
                }
            ]
        }
    ],
    model="gpt-4o",
)

In [85]:
print(response.choices[0].message.content)

The image depicts an animated scene with a penguin in a suit holding a gun and an elephant in a leather jacket in an urban setting.


## Text to speech

In [91]:
with client.audio.speech.with_streaming_response.create(
        model="tts-1-hd",
        voice="nova",
        input="Welcome to my jupyter notebook, folks!",
) as response:
    response.stream_to_file("genai_speech.mp3")